<a href="https://colab.research.google.com/github/Harsh24032000/Fault-Detection-in-Power-Microgrid/blob/master/ANN_Model_Code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [34]:
! pip install keras
! pip install tensorflow

ERROR: Could not find a version that satisfies the requirement scikit (from versions: none)
ERROR: No matching distribution found for scikit


In [39]:
! pip install sklearn
! pip install scikeras

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [40]:
import numpy as np
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from scikeras.wrappers import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [41]:
seed=7
np.random.seed(seed)

In [42]:
df=pd.read_excel('Data.xlsx')
df1=df.values
df

,Current 1,P_L 1,Current 2,P_L 2,Current 3,P_L 3,Current Ideal,P_L Ideal,Time
0,8.858,492.0,5.249,1093.0,1.9690,1640,9.465000e-05,1968,1
1,4.429,246.0,2.625,546.7,0.9844,820,8.514000e-05,984,2
2,4.429,246.0,2.625,546.7,0.9844,820,-9.609000e-11,984,3
3,4.429,246.0,2.625,546.7,0.9844,820,-9.609000e-11,984,4
4,4.429,246.0,2.625,546.7,0.9844,820,-9.609000e-11,984,5
5,7.382,410.0,4.374,911.1,1.6400,1670,5.178000e-06,1640,6
6,14.760,819.9,8.748,1822.0,3.2800,2733,-9.125000e-05,3280,7
7,26.570,1476.0,15.750,3280.0,5.9050,4919,-1.579000e-04,5903,8
8,29.530,1640.0,17.500,3644.0,6.5610,5466,5.177000e-06,6559,9
9,14.760,819.9,8.749,1822.0,3.2810,2733,3.462000e-04,3280,10


In [46]:
po=pd.DataFrame(columns=['current','load','result','Time'])
po

,current,load,result,Time


In [47]:
po = pd.DataFrame(columns=['current', 'load', 'result', 'Time'])

for i in range(23):
    # Create a dictionary for the current row
    row_data = {
        'current': df.at[i, "Current 1"],
        'load': df.at[i, "P_L 1"],
        'result': 1,
        'Time': df.at[i, "Time"]
    }
    
    # Convert the dictionary to a DataFrame
    row_df = pd.DataFrame([row_data])
    
    # Concatenate the row DataFrame to `po`
    po = pd.concat([po, row_df], ignore_index=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_12924\1002947168.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  po = pd.concat([po, row_df], ignore_index=True)


In [54]:
po

,current,load,result,Time
0,8.858,492.0,1,1
1,4.429,246.0,1,2
2,4.429,246.0,1,3
3,4.429,246.0,1,4
4,4.429,246.0,1,5
5,7.382,410.0,1,6
6,14.760,819.9,1,7
7,26.570,1476.0,1,8
8,29.530,1640.0,1,9
9,14.760,819.9,1,10


In [50]:
# for i in range(23):
#     po=po.append({'current':df.at[i,"Current 3"],'load':df.at[i,"P_L 3"],'result':3,'Time':df.at[i,"Time"]},ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [52]:
rows = []

for i in range(23):
    row_data = {
        'current': df.at[i, "Current 1"],
        'load': df.at[i, "P_L 1"],
        'result': 1,
        'Time': df.at[i, "Time"]
    }
    rows.append(row_data)

# Convert the list of dictionaries to a DataFrame
po = pd.DataFrame(rows)

In [53]:
for i in range(23):
    # po=po.append({'current':df.at[i,"Current Ideal"],'load':df.at[i,"P_L Ideal"],'result':0,'Time':df.at[i,"Time"]},ignore_index=True)

AttributeError: 'DataFrame' object has no attribute 'append'

In [55]:
po = pd.DataFrame(columns=['current', 'load', 'result', 'Time'])

for i in range(23):
    # Create a dictionary for the current row
    row_data = {
        'current': df.at[i, "Current Ideal"],
        'load': df.at[i, "P_L Ideal"],
        'result': 0,
        'Time': df.at[i, "Time"]
    }
    
    # Convert the dictionary to a DataFrame
    row_df = pd.DataFrame([row_data])
    
    # Concatenate the row DataFrame to `po`
    po = pd.concat([po, row_df], ignore_index=True)

C:\Users\HP\AppData\Local\Temp\ipykernel_12924\2078162155.py:16: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  po = pd.concat([po, row_df], ignore_index=True)


In [56]:
po

,current,load,result,Time
0,9.465000e-05,1968,0,1
1,8.514000e-05,984,0,2
2,-9.609000e-11,984,0,3
3,-9.609000e-11,984,0,4
4,-9.609000e-11,984,0,5
5,5.178000e-06,1640,0,6
6,-9.125000e-05,3280,0,7
7,-1.579000e-04,5903,0,8
8,5.177000e-06,6559,0,9
9,3.462000e-04,3280,0,10


In [57]:
po = pd.concat([po,pd.get_dummies(po['Time'], prefix='Time',dummy_na=True)],axis=1).drop(['Time'],axis=1)


In [58]:
po

,current,load,result,Time_1.0,Time_2.0,Time_3.0,Time_4.0,Time_5.0,Time_6.0,Time_7.0,...,Time_15.0,Time_16.0,Time_17.0,Time_18.0,Time_19.0,Time_20.0,Time_21.0,Time_22.0,Time_23.0,Time_nan
0,9.465000e-05,1968,0,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,8.514000e-05,984,0,False,True,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,-9.609000e-11,984,0,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,-9.609000e-11,984,0,False,False,False,True,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,-9.609000e-11,984,0,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
5,5.178000e-06,1640,0,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False
6,-9.125000e-05,3280,0,False,False,False,False,False,False,True,...,False,False,False,False,False,False,False,False,False,False
7,-1.579000e-04,5903,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
8,5.177000e-06,6559,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,3.462000e-04,3280,0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [59]:
poo=po.values

In [60]:
yy=poo[:,2]
po.drop(['result'],axis="columns",inplace=True)

In [61]:
XX=poo[:,:]


In [67]:
def neural_net():
    model = Sequential()
    model.add(Dense(16, input_dim=27, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(8, kernel_initializer='normal', activation='relu'))
    model.add(Dense(4, kernel_initializer='normal',activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [68]:
# from keras.utils import np_utils

In [69]:
from keras.utils import to_categorical

In [71]:
encoder = LabelEncoder()
encoder.fit(yy)
encoded_Y = encoder.transform(yy)
# dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y = to_categorical(encoded_Y)

In [72]:
dummy_y

array([[1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]])

In [73]:
dummy_XX=XX

In [74]:
scaler=StandardScaler()

In [75]:
dummy_XX=scaler.fit_transform(dummy_XX)

In [76]:
dummy_XX

array([[-0.33101709, -0.68676898,  0.        ,  4.69041576, -0.21320072,
        -0.21320072, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072,  0.        ],
       [-0.33101816, -1.09187104,  0.        , -0.21320072,  4.69041576,
        -0.21320072, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072,  0.        ],
       [-0.33102779, -1.09187104,  0.        , -0.21320072, -0.21320072,
         4.69041576, -0.21320072, -0.21320072, -0.21320072, -0.21320072,
        -0.21320072, -0.21320072, -0.21320072, -0.2132

In [79]:
# mm=neural_net()
# history=mm.fit(XX,dummy_y,epochs=500)
XX = np.random.rand(100, 10)  # 100 samples, 10 features
dummy_y = np.random.randint(2, size=(100, 1))  # Binary classification labels

# Define the neural network model
mm = Sequential()
mm.add(Dense(64, input_dim=10, activation='relu'))  # Input layer
mm.add(Dense(32, activation='relu'))  # Hidden layer
mm.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
mm.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
history = mm.fit(XX, dummy_y, epochs=500, batch_size=10)

# Access training history
print(history.history)

Epoch 1/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 2s 9ms/step - accuracy: 0.4606 - loss: 0.6967
Epoch 2/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5293 - loss: 0.6950 
Epoch 3/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5343 - loss: 0.6911 
Epoch 4/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6264 - loss: 0.6815 
Epoch 5/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6700 - loss: 0.6785 
Epoch 6/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6086 - loss: 0.6756 
Epoch 7/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6584 - loss: 0.6698 
Epoch 8/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6085 - loss: 0.6807 
Epoch 9/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5274 - loss: 0.6847 
Epoch 10/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5982 - loss: 0.6688 
Epoch 11/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.5745 - loss: 0.6766 
Epoch 12/500
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step -